In [22]:
from pandas_profiling import ProfileReport
from util import get_nutrition_data
import os
import numpy as np
import pandas as pd
from bokeh.plotting import (
    figure,
    show,
    output_notebook,
    ColumnDataSource,
    output_file,
)
from bokeh.models import HoverTool, Span
from bokeh.io import curdoc


In [23]:
nutri_df = get_nutrition_data()
nutri_df.head()

2023-02-27 15:34:21 [info     ] [get_worksheet]                workbook_name=nutrition worksheet_name=nutrition
2023-02-27 15:34:25 [info     ] [get_worksheet]                workbook_name=nutrition worksheet_name=nutrition-manual


,barcode,group,pantry_ingredient,product_name,quantity,brands,completeness,nutriscore_grade,level_fat,level_salt,level_saturated_fat,level_sugars,per_100g_carbohydrates,per_100g_energy_kcal,per_100g_fat,per_100g_fiber,per_100g_proteins,per_100g_salt,per_100g_saturated_fat,per_100g_sugars
0,4046700003756,Dairy products,15% fat cream,LAC Kochsahne,200 g,LAC,0.6875,d,moderate,low,high,low,4.1,163.0,15.0,0.0,2.9,0.11,9.40,4.1
1,4046700004234,Dairy products,30% fat cream,Schlagsahne,200 g,LAC,0.775,d,high,low,high,low,3.3,311.0,32.0,0.0,2.4,0.08,20.20,3.3
2,4008100120341,Sauces,apple cider vinegar,Bio Apfelessig,500ml,Hengstenberg,0.775,NaN,low,low,low,low,0.7,19.0,0.0,0.0,0.0,0,0.00,0.0
3,4311501711699,Canned,applesauce,Apfelmus,360g,Edeka,0.6875,a,low,low,low,moderate,12.5,57.0,0.1,0.0,0.3,0.01,0.05,11.6
4,4311501356517,Canned,artichoke hearts,Artischocken Herzen,240 g,Edeka,0.6875,a,low,moderate,low,low,4.0,27.0,0.0,0.0,1.4,0.8,0.00,1.3


In [24]:
nutri_df.group.value_counts()

Dairy products      22
Sauces              19
Frozen goods        16
Prepared            13
Canned              13
Baking              12
Meats                8
Nuts and seeds       8
Grains               6
Juices               5
Spices and herbs     4
Fats and oils        4
Pasta                4
Beverages            2
Vegetables           2
Fish                 2
Fruits               1
Name: group, dtype: int64

In [25]:
nutri_df[nutri_df.pantry_ingredient.str.contains("peanut")]

,barcode,group,pantry_ingredient,product_name,quantity,brands,completeness,nutriscore_grade,level_fat,level_salt,level_saturated_fat,level_sugars,per_100g_carbohydrates,per_100g_energy_kcal,per_100g_fat,per_100g_fiber,per_100g_proteins,per_100g_salt,per_100g_saturated_fat,per_100g_sugars
92,5060343740099,Canned,peanut butter,Roasted peanut butter,1 kg,Bulk,0.9,a,high,low,high,moderate,12.0,627.0,51.0,4.9,28.0,0,6.3,6.1


In [26]:
ingredient_group_remapping = {
    "canned tuna": "Fish",
    "peanut butter": "Nuts and seeds",
    "Baking": "Grains",
}
for ingr, new_group in ingredient_group_remapping.items():
    nutri_df.loc[
        nutri_df.pantry_ingredient.str.lower() == ingr, "group"
    ] = new_group

In [27]:
# https://www.omnicalculator.com/conversion/grams-to-calories#grams-to-calories-conversion
kcal_per1g_carbohydrates = 4
kcal_per1g_fat = 9
kcal_per1g_proteins = 4
kcal_per_1g_fiber = 1.7  # assumption, average

In [28]:
nutri_df["calc_per_100g_energy_kcal"] = (
    nutri_df["per_100g_carbohydrates"] * kcal_per1g_carbohydrates
    + nutri_df["per_100g_fat"] * kcal_per1g_fat
    + nutri_df["per_100g_proteins"] * kcal_per1g_proteins
    + nutri_df["per_100g_fiber"] * kcal_per_1g_fiber
)

In [29]:
# should double-check anything that differs more than e.g. 10 kCal
nutri_df["calc_diff_energy_kcal"] = (
    nutri_df["per_100g_energy_kcal"] - nutri_df["calc_per_100g_energy_kcal"]
)

nutri_df["rel_saturated_fats"] = (
    nutri_df["per_100g_saturated_fat"] / nutri_df["per_100g_fat"]
)

In [30]:
# daily recommended quantities
# general diet: around 10% from saturated fats
# heart diet: < 5-6 % of daily caloric intake should be from saturated fats
# nhs guidelines: 20/30 g per day (women/man)

daily_calories = {"alex": 2700, "ariel": 1800}

daily_max_guidelines_absolute = {
    "trans_fats_g": 5,
}

daily_max_guidelines_relative = {
    "saturated_fats_alex": 5,
    "saturated_fats_ariel": 7,
    "overall_fats": 35,
}

daily_max_guidelines_absolute["saturated_fats_alex"] = (
    daily_max_guidelines_relative["saturated_fats_alex"]
    / 100
    * daily_calories["alex"]
    / kcal_per1g_fat
)
daily_max_guidelines_absolute["saturated_fats_ariel"] = (
    daily_max_guidelines_relative["saturated_fats_ariel"]
    / 100
    * daily_calories["ariel"]
    / kcal_per1g_fat
)
daily_max_guidelines_absolute["overall_fats_alex"] = (
    daily_max_guidelines_relative["overall_fats"]
    / 100
    * daily_calories["alex"]
    / kcal_per1g_fat
)
daily_max_guidelines_absolute["overall_fats_ariel"] = (
    daily_max_guidelines_relative["overall_fats"]
    / 100
    * daily_calories["ariel"]
    / kcal_per1g_fat
)

print(daily_max_guidelines_absolute)

{'trans_fats_g': 5, 'saturated_fats_alex': 15.0, 'saturated_fats_ariel': 14.000000000000002, 'overall_fats_alex': 104.99999999999999, 'overall_fats_ariel': 70.0}


## Summary plot

In [31]:
curdoc().theme = "light_minimal"

In [32]:
sel_df = nutri_df[
    nutri_df.group.isin(["Dairy products", "Nuts and seeds", "Meats", "Fish"])
]

color_map = {
    "Dairy products": "green",
    "Nuts and seeds": "yellow",
    "Meats": "red",
    "Fish": "blue",
}

In [33]:
sel_df.loc[:, "marker_size"] = 35 * sel_df["rel_saturated_fats"]
sel_df.loc[:, "marker_color"] = sel_df["group"].map(color_map)

ds = ColumnDataSource(data=sel_df)

tooltips = [("(x,y)", "($x, $y)"), ("ingredient", "@pantry_ingredient")]

p = figure(
    title="Summary chart",
    x_axis_label="Energy per 100 g (kCal)",
    y_axis_label="Fat content per 100 g (g)",
    height=800,
    width=1600,
    tooltips=tooltips,
)

circles = p.circle(
    "per_100g_energy_kcal",
    "per_100g_fat",
    color="marker_color",
    size="marker_size",
    # legend="group",
    source=ds,
)
p.legend.location = "top_left"
p.legend.click_policy = "hide"

p.text(
    sel_df["per_100g_energy_kcal"],
    sel_df["per_100g_fat"] + 5,
    text=sel_df["pantry_ingredient"],
    text_color="white",
    text_align="center",
    text_font_size="8pt",
)

fat_line_alex = Span(
    location=daily_max_guidelines_absolute["overall_fats_alex"],
    dimension="width",
    line_color="red",
    line_width=3,
)
fat_line_ariel = Span(
    location=daily_max_guidelines_absolute["overall_fats_ariel"],
    dimension="width",
    line_color="blue",
    line_width=3,
)

p.renderers.extend([fat_line_ariel, fat_line_alex])

output_notebook()
show(p)

/var/folders/_k/zsk1y9vd1xlgsstl24phb7tc0000gn/T/ipykernel_35194/2716318597.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sel_df.loc[:, "marker_size"] = 35 * sel_df["rel_saturated_fats"]
/var/folders/_k/zsk1y9vd1xlgsstl24phb7tc0000gn/T/ipykernel_35194/2716318597.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sel_df.loc[:, "marker_color"] = sel_df["group"].map(color_map)
/var/folders/_k/zsk1y9vd1xlgsstl24phb7tc0000gn/T/ipykernel_35194/2716318597.py:25: UserWarning: 
You are attempting to set `plot

Loading BokehJS ...

## Sat fat vs. fat chart

In [34]:
nutri_df.group.value_counts()


Dairy products      22
Sauces              19
Frozen goods        16
Prepared            13
Baking              12
Canned              11
Nuts and seeds       9
Meats                8
Grains               6
Juices               5
Spices and herbs     4
Fats and oils        4
Pasta                4
Fish                 3
Beverages            2
Vegetables           2
Fruits               1
Name: group, dtype: int64

In [35]:
sel_df = nutri_df[
    nutri_df.group.isin(
        [
            "Dairy products",
            "Nuts and seeds",
            "Meats",
            "Fish",
            "Grains",
            "Fats and oils",
            "Canned",
        ]
    )
]

color_map = {
    "Dairy products": "green",
    "Nuts and seeds": "black",
    "Meats": "red",
    "Fish": "blue",
    "Grains": "grey",
    "Fats and oils": "orange",
    "Canned": "purple",
}

In [36]:
show_text = True

sel_df.loc[:, "marker_size"] = 0.8 * sel_df["per_100g_proteins"]
sel_df.loc[:, "marker_color"] = sel_df["group"].map(color_map)

ds = ColumnDataSource(data=sel_df)

tooltips = [
    ("(x,y)", "($x, $y)"),
    ("ingredient", "@pantry_ingredient"),
    ("group", "@group"),
    ("kCal per 100g", "@per_100g_energy_kcal"),
    ("fat per 100g", "@per_100g_fat"),
    ("sat. fat per 100g", "@per_100g_saturated_fat"),
    ("proteins per 100g", "@per_100g_proteins"),
]

p = figure(
    title="Summary chart",
    x_axis_label="Fat (g) per 100 g",
    y_axis_label="Saturated fat (g) per 100 g",
    height=800,
    width=1600,
    tooltips=tooltips,
)

for group_name, group_df in sel_df.groupby("group"):
    circles = p.circle(
        "per_100g_fat",
        "per_100g_saturated_fat",
        color="marker_color",
        # size="marker_size",
        size=0,
        legend_label=group_name,
        source=ColumnDataSource(data=group_df),
    )

    if show_text:
        p.text(
            group_df["per_100g_fat"],
            group_df["per_100g_saturated_fat"] + 0.5,
            text=group_df["pantry_ingredient"],
            text_color=group_df["marker_color"].iloc[0],
            text_align="center",
            angle=-np.radians(60),
            text_font_size="10pt",
            legend_label=group_name,
        )


p.legend.location = "top_left"
p.legend.click_policy = "hide"

fat_line_alex = Span(
    location=daily_max_guidelines_absolute["saturated_fats_alex"],
    dimension="width",
    line_color="red",
    line_width=3,
)
p.text(
    [10],
    [daily_max_guidelines_absolute["saturated_fats_alex"]],
    text="Daily saturated fat limit Alex",
    text_color="red",
    text_align="center",
    text_font_size="8pt",
)

fat_line_ariel = Span(
    location=daily_max_guidelines_absolute["saturated_fats_ariel"],
    dimension="width",
    line_color="blue",
    line_width=3,
)
p.text(
    10,
    daily_max_guidelines_absolute["saturated_fats_ariel"],
    text="Daily saturated fat limit Ariel",
    text_color="blue",
    text_align="center",
    text_font_size="8pt",
)


p.renderers.extend([fat_line_ariel, fat_line_alex])

# output_notebook()
# output_file("sat_vs_overall_fats.html")
show(p)


/var/folders/_k/zsk1y9vd1xlgsstl24phb7tc0000gn/T/ipykernel_35194/1252608894.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sel_df.loc[:, "marker_size"] = 0.8 * sel_df["per_100g_proteins"]
/var/folders/_k/zsk1y9vd1xlgsstl24phb7tc0000gn/T/ipykernel_35194/1252608894.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sel_df.loc[:, "marker_color"] = sel_df["group"].map(color_map)


In [37]:
sel_df.head(2)

,barcode,group,pantry_ingredient,product_name,quantity,brands,completeness,nutriscore_grade,level_fat,level_salt,...,per_100g_fiber,per_100g_proteins,per_100g_salt,per_100g_saturated_fat,per_100g_sugars,calc_per_100g_energy_kcal,calc_diff_energy_kcal,rel_saturated_fats,marker_size,marker_color
0,4046700003756,Dairy products,15% fat cream,LAC Kochsahne,200 g,LAC,0.6875,d,moderate,low,...,0.0,2.9,0.11,9.4,4.1,163.0,0.0,0.626667,2.32,green
1,4046700004234,Dairy products,30% fat cream,Schlagsahne,200 g,LAC,0.775,d,high,low,...,0.0,2.4,0.08,20.2,3.3,310.8,0.2,0.631250,1.92,green


In [38]:
display(
    sel_df[
        [
            "pantry_ingredient",
            "group",
            "per_100g_fat",
            "per_100g_saturated_fat",
        ]
    ].sort_values(["per_100g_fat", "per_100g_saturated_fat"], ascending=False)
)


,pantry_ingredient,group,per_100g_fat,per_100g_saturated_fat
124,sunflower oil,Fats and oils,92.0,10.00
87,olive oil,Fats and oils,91.0,16.00
93,pecans,Nuts and seeds,72.0,6.20
131,walnuts,Nuts and seeds,71.2,6.90
62,hazelnuts,Nuts and seeds,62.4,4.50
...,...,...,...,...
133,whole tomatoes,Canned,0.2,0.00
3,applesauce,Canned,0.1,0.05
107,roasted red bell pepper,Canned,0.1,0.03
21,chicken stock,Canned,0.1,0.00


In [50]:
sort_fields = {
    "Dairy products": "per_100g_fat",
    "Nuts and seeds": "per_100g_fat",
    "Meats": "per_100g_fat",
    "Fish": "per_100g_fat",
    "Grains": "per_100g_carbohydrates",
    "Fats and oils": "per_100g_fat",
    "Canned": "per_100g_fat",
    "Prepared": "per_100g_carbohydrates",
}

rename_fields = {
    "pantry_ingredient": "Ingredient (100 g)",
    "per_100g_energy_kcal": "Energy (kCal)",
    "per_100g_fat": "Fat (g)",
    "per_100g_saturated_fat": "Saturated fat (g)",
    "per_100g_proteins": "Protein (g)",
    "per_100g_salt": "Salt (g)",
    "per_100g_carbohydrates": "Carbs (g)",
}

In [52]:
sel_df = nutri_df.copy()

sel_df.loc[
    sel_df.pantry_ingredient.str.contains("flour", case=False)
    | sel_df.pantry_ingredient.str.contains("starch", case=False),
    "group",
] = "Grains"
sel_df.loc[sel_df.group == "Fish", "group"] = "Meats"

group_display_mapping = {"Grains": "Grains & flours", "Meats": "Meats & fish"}

out_file = "nutrition_tables.xlsx"
sheet_name = "nutrition"

if os.path.exists(out_file):
    os.remove(out_file)

row_count = 1

with pd.ExcelWriter(
    out_file,
    engine="xlsxwriter",
) as writer:

    for group in sort_fields.keys():
        df_here = sel_df[sel_df.group == group]

        if df_here.shape[0] == 0:
            continue

        df_print = (
            df_here[
                [
                    "pantry_ingredient",
                    "per_100g_energy_kcal",
                    "per_100g_fat",
                    "per_100g_saturated_fat",
                    "per_100g_proteins",
                    "per_100g_carbohydrates",
                    "per_100g_salt",
                ]
            ]
            .round(2)
            .sort_values(sort_fields[group], ascending=False)
            .rename(columns=rename_fields)
        )
        out_df = df_print.style.background_gradient(cmap="Greys", axis=0).hide(
            axis="index"
        )

        out_df.to_excel(
            writer,
            sheet_name=sheet_name,
            startrow=row_count,
            startcol=0,
            index=False,
        )
        sheet = writer.sheets[sheet_name]
        # cell_format = sheet.add_format()
        # cell_format.set_bold()
        sheet.write_string(
            row_count - 1,
            0,
            f"{group_display_mapping.get(group, group)} (sort: {rename_fields[sort_fields[group]]})",
        )

        row_count += df_print.shape[0] + 5
